## Step 2: Run visibility fit by forward-modeling with RADMC-3D (ray tracing) and GALARIO (for FFT) through emcee package

Review what the belt looks like, and use it to inform the choice of parameters and the setup for the fit. <br> Start this script in the target's main directory (workingdir+'/'+sourcetag) from Step 1.

In [ ]:
#Read in directories as defined in step 1
radmcgalapath, casapath, sourcetag, workingdir, vis, nvis = pickle.load(open('./imagepars.npy','rb'))
#Open CASA viewer to display the belt. Keep display open to inform your choice of parameters below.
os.chdir('../imaging')
!{casapath}/casaviewer &

Define a few FIXED parameters for the fit

In [ ]:
Lstar=0.111  #Stellar luminosity, in Solar luminosities, for blackbody temperature calculation
dist=14.7  #distance to the star, in pc
imsize=24.0     #size (diameter) of the image you want to calculate the radiative transfer over, in arcsec.
#Make sure it includes all emission from the disk, but be wary of making it too large as ray-tracing becomes too expensive

Define disk starting parameters, and priors. This is for a belt with radially Gaussian surface density distribution,
and vertically Gaussian number density distribution, e.g. Matra et al. 2019b, Sect. 4.2, and Eq. 1. <br>
$$ \rho=\Sigma_0\ e^{-\frac{(r-r_{\rm c})^2}{2\sigma^2}}\frac{e^{-\frac{z^2}{2(hr)^2}}}{\sqrt{2\pi}hr} $$<br>
*Note that rho_0 in Eq. of the paper should really be in surface density units, so should have been represented as a Sigma_0 in the paper.* <br>
*Also note this is NOT the same as a belt with a radial density, rather than surface density, distribution, in which the sqrt{2pi}hr normalization would not be present*

In [2]:
#Flux scaling
fluxdensity=1.8e-3 #Total flux density of the belt, in Jy
fluxdensity_dwn, fluxdensity_up = [0.1e-3, 50e-3] #Lower and upper bounds on its prior

#Radial parameters
rmid=99.0/dist # Radial peak location of Gaussian surface density, arcsec
rmid_dwn, rmid_up = [0.3, 12.0]
sigma=32.0/2.35/dist #standard deviation of radial Gaussian surface density
sigma_dwn, sigma_up = [0.05, 'halfrmid'] #halfrmid is special setup that enables the maximum sigma to be half whatever 
#rmid is in current model evaluation.

#Vertical parameters
useh=True  #Do you want to leave the vertical aspect ratio as a free parameter? Only worth it for belts that you 
#believe have a chance to be vertically resolved
if useh:
    h=0.05 #aspect ratio of belt, constant with radius according to the equation above.
    h_dwn, h_up = [0.005, 0.2]
else:
    h=0.03

#Geometry parameters
incl=65.0 #inclination, degrees from face-on
incl_dwn, incl_up = [0.1, 89.99]
posang=5.5 #position angle, East of North. **Needs to be between 0 and 180 degrees!**
#Here below set priors in a way that your position angle is not at the edge of the prior distribution. 
#No need to modify these priors.
if posang<45.0 or posang>135.0:
    posang_dwn, posang_up = [-90.0, 90.0]
else:
    posang_dwn, posang_up = [0.0, 180.0]
    
#Add star if wanted/needed. This will be located at the geometrical center of the belt.
star=True
if star:
    fstar=4e-5 #star's flux density, in Jy
    fstar_dwn, fstar_up = [1e-6, 2e-3]

IndentationError: unexpected indent (<ipython-input-2-19407f8b6146>, line 132)

Define dataset-dependent parameters: the offsets of the (star+)belt system from the phase center of the observation (dRA and dDec), and the weight factor by which to multiply all visibility weights of each dataset. <br>
The latter is necessary to ensure the measurement uncertainty on each visibility is correct, and therefore that the uncertainties derived on the parameters of the fit are correct. This is not very expensive to do as the values of these weight factors converge very quickly.

In [ ]:
if nvis>1: 
    #If you have more than one visibility dataset to fit, define lists - rather than values - of starting 
    #parameters and priors
    dRA=[-0.07,0.1,0.1] #RA offset of star+disk from phase center of observations. 
    ###Remember, RA is positive to the right in the CASA viewer image!###
    dRA_dwn=[-3.0,-3.0,-3.0]
    dRA_up=[3.0,3.0,3.0]
    dDec=[0.14,-0.1,-0.1] #Dec offset of star+disk from phase center of observations
    dDec_dwn=[-3.0,-3.0,-3.0]
    dDec_up=[3.0,3.0,3.0]
    wtfact=[0.289,0.289,1e-4] #factors by which incorrect weights should be multiplied by.
    #This is typically a value between 0.2 and 0.8 for later ALMA Cycles (>~3, but depends on the CASA version used for
    #reduction), around 1 (i.e. weights should be correct) for SMA data. For early ALMA Cycles, this factor comes out 
    #very low, something between 1e-5 and a few times 1e-4 (i.e. weights are largely overestimated by CASA).
    wtfact_dwn=[1e-6,1e-6,1e-6]
    wtfact_up=[10.0,10.0,10.0]
else:
    dRA=[-0.07] #RA offset of star+disk from phase center of observations
    dRA_dwn=[-3.0]
    dRA_up=[3.0]
    dDec=[0.14] #Dec offset of star+disk from phase center of observations
    dDec_dwn=[-3.0]
    dDec_up=[3.0]
    wtfact=[0.289] #factors by which incorrect weights should be multiplied by
    wtfact_dwn=[1e-6]
    wtfact_up=[10.0]

Here below add background sources (galaxies?) that happen to lie in the field and can affect the modelling. <br>
There can be more than 1, and these can be point sources or resolved. If you deem them to be resolved, set resolved=True and model them as a simple inclined 2D Gaussian. <br>
Note that the number of parameters quickly increases if you have many resolved galaxies! Which will slow down your fit.

In [3]:
ngal=0 #Number of galaxies in the field to be modelled.
if ngal>=1:
    resolved=[True, False, False] # if deemed resolved, use 2D Gaussian as galaxy model (6 free parameters), and set 
    #resolved=True for that galaxy. 
    #Otherwise, use point source (3 free parameters) by setting resolved=False.
    fbkg=[285e-6,500e-6,500e-6] #Total flux density of galaxy (galaxies) (Jy)
    fbkg_dwn=[1e-6,1e-6,1e-6]
    fbkg_up=[10e-3,10e-3,10e-3]
    dRAbkg=[2.97,2.97,2.97] # RA offset of galaxy (galaxies) from center of star+disk system. Note that this WILL NOT
    #account for proper motion of the star+disk between datasets taken at different times, 
    #since the offset is fixed for all datasets.
    dRAbkg_dwn=[x-3.0 for x in dRAbkg]
    dRAbkg_up=[x+3.0 for x in dRAbkg]
    dDecbkg=[1.81,1.81,1.81] # Dec offset of galaxy (galaxies) from center of star+disk system. Note that this WILL NOT
    #account for proper motion of the star+disk between datasets taken at different times, 
    #since the offset is fixed for all datasets.
    dDecbkg_dwn=[x-3.0 for x in dDecbkg]
    dDecbkg_up=[x+3.0 for x in dDecbkg]
    sigmagal=[0.3,0,0] # standard deviation (arcsec) of Gaussian used to describe the bkg source
    sigmagal_dwn=[0.05,0.05,0.05]
    sigmagal_up=[1.0,1.0,1.0]
    PAgal=[28.0,0,0] # PA (deg), East of North. **Needs to be between 0 and 180 degrees!**
    #Set prior bounds on PA in a smart way, as done for the disk.
    PAgal_dwn=[]
    PAgal_up=[]
    for i in PAgal:
        if PAgal<45.0 or PAgal>135.0:
            PAgal_dwn.append(-90.0)
            PAgal_up.append(90.0)
        else:
            PAgal_dwn.append(0.0)
            PAgal_up.append(180.0)   
    incgal=[49.0,0,0] # inclination of the bkg source from face-on (degrees)
    incgal_dwn=[0.1,0.1,0.1]
    incgal_up=[89.99,89.99,89.99]
else: 
    resolved=None
    fbkg=None
    dRAbkg=None
    dDecbkg=None
    sigmagal=None
    PAgal=None
    incgal=None

IndentationError: unexpected indent (<ipython-input-3-5a62748f8f4e>, line 77)

Create needed folders for fitting, check that the imaging has been carried out as we need the primary beam for fitting.

In [ ]:
if not os.path.exists(workingdir+'/'+sourcetag+'/imaging'):
    sys.exit('Carry out imaging first: need .pb primary beam image')
for i in ['uvfit']:
    if not os.path.exists(workingdir+'/'+sourcetag+'/'+i):
        os.mkdir(workingdir+'/'+sourcetag+'/'+i)
!cp -r {radmcgalapath}/utils/setup_mcmc.py {workingdir}/{sourcetag}/uvfit/. 
!cp -r {radmcgalapath}/utils/problem_setup_cont_gauss.py {workingdir}/{sourcetag}/uvfit/.  
!cp -r {radmcgalapath}/utils/dustkappa_10445.micr.inp {workingdir}/{sourcetag}/uvfit/. 

Set up model (including ray-tracing with RADMC-3D, and visibility computation with galario) and MCMC run through emcee. The run will be saved at every step at the backend 
address specified. <br>
**IMPORTANT**: setting newbackend to True overwrites existing backend at the given name. If you care about a given MCMC 
that has taken you a long time to run, make sure you do NOT overwrite this backend! <br>
If you set newbackend to False, the MCMC will
continue from its last step. This is what you want to do if you decided to stop the MCMC while it was running, and you 
want to continue it (see further steps below).

In [ ]:
os.chdir(workingdir+'/'+sourcetag+'/uvfit')

#Decide whether you want to overwrite backend (newbackend=True), or continue from existing backend.
newbackend=True
backendaddress='backend_'+sourcetag+'_todaysdate_computersname_v2.pkl'
pickle.dump(backendaddress, open('backendaddress.npy', 'wb'), protocol=2)

%run -i setup_mcmc

And now run MCMC for whichever number of steps (nsteps) you like. <br>
A progress bar will come up if you have installed the tqdm package. If tired of waiting at any point, you can kill it by interrupting the notebook's kernel. The current state of the sampler will be saved within the backend and can be recovered.

In [ ]:
nsteps=1000
sampler.run_mcmc(pos,nsteps,progress=True)

Quick check on how things went. Burnin is the number of steps you're going to eventually throw out because the chains had not converged yet - you can guess that value for now, as long as it's below the number of steps the MCMC ran for so far.

In [ ]:
chain=backend.get_chain()
for j in range(chain.shape[2]):
    pl.figure()
    for i in range(chain.shape[1]):
        pl.plot(chain[:,i,j], alpha=0.1)

If chains look converged for every parameter (you can also use more formal convergence criteria - see e.g. emcee 
documentation), you can be happy and proceed to the next notebook describing postprocessing (Step 3). <br>
If you're not happy, run for longer using:

In [ ]:
# If number of steps is insufficient, i.e. if the chains have not converged for each parameter, run for more steps using:
newbackend=False
%run -i setup_mcmc
nsteps=1000-chain.shape[0] #chain.shape[0] contains the number of steps we have currently run for, and nsteps will be
# the number of steps you want to go further by.
sampler.run_mcmc(None,nsteps,progress=True)